# Optimizing Bark Inference in 🤗 Transformers

The Bark model has been available in 🤗 Transformers since v4.31.0 onwards!

In this notebook, I'll demonstrate how you can optimize Bark inference, based on simple optimizations, and using solely the 🤗 Transformers, 🤗 Optimum and 🤗 Accelerate libraries!

You can play around with Bark and discover it's abilities [here](https://colab.research.google.com/drive/1dWWkZzvu7L9Bunq9zvD-W02RFUXoW-Pd?usp=sharing).

This notebook is organized as follows:

1.   A [reminder](https://colab.research.google.com/drive/1XO0RhINg4ZZCdJJmPeJ9lOQs98skJ8h_#scrollTo=MHjFuiSd_SH3&line=15&uniqifier=1) of bark architecture
2.   An [overview](https://colab.research.google.com/drive/1XO0RhINg4ZZCdJJmPeJ9lOQs98skJ8h_#scrollTo=nJ0_mfi6UTQU&line=3&uniqifier=1) of different optimization techniques and their advantages
3.   A [presentation](https://colab.research.google.com/drive/1XO0RhINg4ZZCdJJmPeJ9lOQs98skJ8h_#scrollTo=QFJOft1JaKBS&line=4&uniqifier=1) of benchmark results

But first, let's prepare the environment!



## Prepare the Environment

Let’s make sure we’re connected to a GPU to run this notebook. To get a GPU, click Runtime -> Change runtime type, then change Hardware accelerator from None to GPU. We can verify that we’ve been assigned a GPU and view its specifications through the nvidia-smi command:

In [1]:
!nvidia-smi

Tue Aug 22 18:09:06 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.99                 Driver Version: 536.99       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090      WDDM  | 00000000:01:00.0  On |                  Off |
|  0%   51C    P8              32W / 450W |    818MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

We see here that we've got on Tesla T4 16GB GPU, although this may vary for you depending on GPU availablity and Colab GPU assignment.

Next, we install the 🤗 Transformers, 🤗 Optimum and 🤗 Accelerate packages from the main branch:

In [2]:
!pip install --upgrade --quiet pip
!pip install --quiet git+https://github.com/huggingface/transformers.git
!pip install --quiet git+https://github.com/huggingface/optimum.git
!pip install --quiet git+https://github.com/huggingface/accelerate.git

ERROR: To modify pip, please run the following command:
C:\Users\aleks\anaconda3\envs\BIGAI\python.exe -m pip install --upgrade --quiet pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
huggingsound 0.1.2 requires datasets<2.0.0,>=1.18.3, but you have datasets 2.14.1 which is incompatible.
huggingsound 0.1.2 requires librosa<0.9.0,>=0.8.1, but you have librosa 0.10.0 which is incompatible.
huggingsound 0.1.2 requires llvmlite<0.37.0,>=0.36.0, but you have llvmlite 0.40.1 which is incompatible.
huggingsound 0.1.2 requires numba<0.54.0,>=0.53.1, but you have numba 0.57.0 which is incompatible.
huggingsound 0.1.2 requires torch<2.0.0,>=1.9.0, but you have torch 2.0.0+cu118 which is incompatible.
spacy-transformers 1.1.4 requires transformers<4.16.0,>=3.4.0, but you have transformers 4.33.0.dev0 which is incompatible.




# 1. Bark Architecture


Bark is a transformer-based text-to-speech model proposed by Suno AI in [suno-ai/bark](https://github.com/suno-ai/bark).

Bark is made of 4 main models:

- `BarkSemanticModel` (also referred to as the 'text' model): a causal auto-regressive transformer model that takes as input tokenized text, and predicts semantic text tokens that capture the meaning of the text.
- `BarkCoarseModel` (also referred to as the 'coarse acoustics' model): a causal autoregressive transformer, that takes as input the results of the `BarkSemanticModel` model. It aims at predicting the first two audio codebooks necessary for EnCodec.
- `BarkFineModel` (the 'fine acoustics' model), this time a non-causal autoencoder transformer, which iteratively predicts the last codebooks based on the sum of the previous codebooks embeddings.
- having predicted all the codebook channels from the `EncodecModel`, Bark uses it to decode the output audio array.




## Load the Model and its Processor

The pre-trained Bark small and large checkpoints can be loaded from the [pre-trained weights](https://huggingface.co/suno/bark) on the Hugging Face Hub. You can change the repo-id with the checkpoint size that you wish to use.

We'll default to the small checkpoint, to keep it fast. But you can try the large checkpoint by using `"suno/bark"` instead of `"suno/bark-small"`.

In [3]:
from transformers import BarkModel

model = BarkModel.from_pretrained("suno/bark-small")

Place the model to an accelerator device. Don't forget to connect to a GPU if you use colab, or to use a GPU if you are on your own hardware!

In [4]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)

Load the processor, which will takes care of tokenization and optional speaker embeddings.

In [5]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("suno/bark-small")

# 2. Optimization techniques

Thanks to 🤗 Transformers off-the-shelves optimization features and to 🤗 HuggingFace galaxies of libraries (🤗 Optimum and 🤗 Accelerate), you can easily use features that will make your life easier.


# Some set-ups

Let's prepare the inputs and define a function to measure the latency and GPU memory footprint of the Bark generation method.

In [6]:
# first, prepare the inputs
text_prompt = "Let's try generating speech, with Bark, a text-to-speech model"
inputs = processor(text_prompt).to(device)

Measuring the latency and GPU memory footprint requires the use of Cuda methods to get an accurate picture of these metrics!

In [7]:
import torch
from transformers import set_seed


# Pepare an utility function for latency and GPU memory footprint measurement
def measure_latency_and_memory_use(model, inputs, nb_loops = 1):

  # define Events that measure start and end of the generate pass
  start_event = torch.cuda.Event(enable_timing=True)
  end_event = torch.cuda.Event(enable_timing=True)

  # reset cuda memory stats and empty cache
  torch.cuda.reset_peak_memory_stats(device)
  torch.cuda.empty_cache()
  torch.cuda.synchronize()

  # get the start time
  start_event.record()

  # actually generate
  for _ in range(nb_loops):
        # set seed for reproductibility
        set_seed(0)
        output = model.generate(**inputs, do_sample = True, fine_temperature = 0.4, coarse_temperature = 0.8)

  # get the end time
  end_event.record()
  torch.cuda.synchronize()

  # measure memory footprint and elapsed time
  max_memory = torch.cuda.max_memory_allocated(device)
  elapsed_time = start_event.elapsed_time(end_event) * 1.0e-3

  print('Execution time:', elapsed_time/nb_loops, 'seconds')
  print('Max memory footprint', max_memory*1e-9, ' GB')

  return output

## Base case

Before actually talk about optimization technique, let's listen to a little example generated by the non-optimized model and measure how long it takes to generate.

In [8]:
# now let's measure how long it takes to generate with 5 iteration of the same sample with the base model

with torch.inference_mode():
  speech_output = measure_latency_and_memory_use(model, inputs, nb_loops = 5)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generati

Execution time: 10.218284375 seconds
Max memory footprint 1.914612224  GB


In [10]:
from IPython.display import Audio

# now, listen to the output
sampling_rate = model.generation_config.sample_rate
Audio(speech_output[0].cpu().numpy(), rate=sampling_rate)

### Important note:

 Here, the number of iteration is actually quite low. To accurately measure and compare results, one should increase the number of iterations!

One of the main reasons for the importance of increasing `nb_loops` is that the speech lengths generated vary greatly between different iterations, even with a fixed input.

 One consequence of this is that the latency measured by `measure_latency_and_memory_use` may not actually reflect the actual performance of optimization techniques! Only the benchmark of the end of the notebook should be trusted!



## 1. 🤗 Bettertransformer

Bettertransformer is an 🤗 Optimum feature that performs kernel fusion under the hood. This means that certain model operations will be better optimized on the GPU and that the model will be ultimately faster.

Want to know more? See this [blog post](https://medium.com/pytorch/bettertransformer-out-of-the-box-performance-for-huggingface-transformers-3fbe27d50ab2).

🤗 bettertransformer is available with just two additional lines of code.



In [28]:
from optimum.bettertransformer import BetterTransformer

# Use bettertransform for flash attention
model = BetterTransformer.transform(model, keep_original_model=False)

Exception: `BetterTransform.transform()` was called on a model already using Better Transformer modeling.

In [29]:
# measure and listen to the output

with torch.inference_mode():
  speech_output = measure_latency_and_memory_use(model, inputs, nb_loops = 5)


Audio(speech_output[0].cpu().numpy(), rate=sampling_rate)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generati

Execution time: 5.24703125 seconds
Max memory footprint 0.45102592  GB


**What does it bring to the table?**

There's no performance degradation, which means you can get exactly the same result as without this function, while gaining 20% to 30% in speed!



## 2. Half-precision

Most AI models typically use a storage format called single-precision floating point, i.e `fp32`. What does it means in practice ? Each number is stored using 32 bits.

You can thus chose to encode the numbers using 16 bits, with what is called half-precision floating point, i.e `fp16`, and use half as many storage than before! More than that, you also get inference speed-up!

Of course, it also comes with a small performance degradation, since operations inside the model won't be as precise as using `fp32`.

You can load a 🤗 Transformers model with half-precision by simpling adding `torch_dtype=torch.float16` to the `BarkModel.from_pretrained(...)` line!

In other words:



In [13]:
model = BarkModel.from_pretrained("suno/bark-small", torch_dtype=torch.float16).to(device)

In [14]:
# measure and listen to the output

with torch.inference_mode():
  speech_output = measure_latency_and_memory_use(model, inputs, nb_loops = 5)


Audio(speech_output[0].cpu().numpy(), rate=sampling_rate)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generati

Execution time: 9.304853125 seconds
Max memory footprint 2.7755299840000003  GB


**What does it bring to the table?**

With a slight degradation in performance, you benefit from a memory footprint reduced by 50% and a speed gain of 5%.

## 3. CPU offload

As mentioned in the first section of this booklet, Bark is made up of 4 sub-models, which are called up sequentially during audio generation. **In other words, while one sub-model is in use, the other sub-models are idle.**

Why is this a problem? GPU memory is precious in AI, because it's where operations are fastest, and it's often a bottleneck.

A simple solution is to unload sub-models from the GPU when they are inactive. This operation is called CPU offload.

**Good news:** CPU offload for Bark was integrated into 🤗 Transformers and you can use it with only one line of code.

You only need to make sure 🤗 Accelerate is installed!




In [15]:
model = BarkModel.from_pretrained("suno/bark-small")

# Enable CPU offload
model.enable_cpu_offload()

In [16]:
# measure and listen to the output

with torch.inference_mode():
  speech_output = measure_latency_and_memory_use(model, inputs, nb_loops = 5)


Audio(speech_output[0].cpu().numpy(), rate=sampling_rate)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generati

Execution time: 11.286925 seconds
Max memory footprint 2.6311982080000003  GB


**What does it bring to the table?**

With a slight degradation in speed (10%), you benefit from a huge memory footprint reduction (60% 🤯).

With this feature enabled, `bark-large` footprint is now only 2GB instead of 5GB.
That's the same memory footprint as `bark-small`!
Want more? With `fp16` enabled, it's even down to 1GB.

## 4. Combine

Let's bring it all together. The good news is that you can combine optimization techniques, which means you can use CPU offload, as well as half-precision and 🤗 bettertransformer!

In [17]:
# load in fp16
model = BarkModel.from_pretrained("suno/bark-small", torch_dtype=torch.float16).to(device)

# convert to bettertransformer
model = BetterTransformer.transform(model, keep_original_model=False)

# enable CPU offload
model.enable_cpu_offload()

The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


In [18]:
# measure and listen to the output

with torch.inference_mode():
  speech_output = measure_latency_and_memory_use(model, inputs, nb_loops = 5)


Audio(speech_output[0].cpu().numpy(), rate=sampling_rate)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generati

Execution time: 8.894178906250001 seconds
Max memory footprint 2.1926906880000003  GB


**What does it bring to the table?**

**Ultimately, you get a 23% speed-up and a huge 80% memory saving!**

## Using batching

Want more?

Altogether, the 3 optimization techniques bring even better results when batching.
Batching means combining operations for multiple samples to bring the overall time spent generating the samples lower than generating sample per sample.

Here is a quick example on how you can use it:


In [27]:
text_prompt = [
    "Could you say that again, please?"]

inputs = processor(text_prompt).to(device)


with torch.inference_mode():
  # samples are generated all at once
  speech_output = model.generate(**inputs, do_sample = True, fine_temperature = 0.4, coarse_temperature = 0.8)



# let's listen to the output samples!
Audio(speech_output[0].cpu().numpy(), rate=sampling_rate)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


# 3. Benchmark results

As mentioned above, the little experiment we've carried out is an exercise in thinking and needs to be extended for a better measure of performance. One also needs to warm-up the GPU with a few blank iterations before properly measuring performance.

Here are the results of a 100-sample benchmark extending the measurements, **using the large version of Bark**.

The benchmark was run on an NVIDIA TITAN RTX 24GB with a maximum of 256 new tokens.

## How to read the results?

### Latency

It measures the duration of a single call to the generation method, regardless of batch size.

In other words, it's equal to $\frac{elapsedTime}{nbLoops}$.

**A lower latency is prefered.**

### Maximum memory footprint

It measures the maximum memory used during a single call to the generation method.

**A lower footprint is prefered.**


### Throughput

It measures the number of samples generated per second. This time, batch size is taken into account.

In other words, it's equal to $\frac{nbLoops*batchSize}{elapsedTime}$.

**A higher footprint is prefered.**



## No batching

Here are the results with `batch_size=1`.



| Absolute values             | Latency | Memory  |
|-----------------------------|---------|---------|
| no optimization             |   10.48 | 5025.0M |
| bettertransformer only      |    7.70 | 4974.3M |
| offload + bettertransformer |    8.90 | 2040.7M |
| offload + bettertransformer + fp16            |    8.10 | 1010.4M |

| Relative value              | Latency | Memory |
|-----------------------------|---------|--------|
| no optimization             |      0% |     0% |
| bettertransformer only      |    -27% |    -1% |
| offload + bettertransformer |    -15% |   -59% |
| offload + bettertransformer + fp16            |    -23% |   -80% |

### Comment

As expected, CPU offload greatly reduces memory footprint while slightly increasing latency.

However, combined with bettertransformer and `fp16`, we get the best of both world, huge latency and memory decrease !

## Batch size set to 8
And here are the benchmark results but with `batch_size=8` and throughput measurement.

Note that since `bettertransformer` is a free optimization because it does exactly the same operation and has the same memory footprint as the non-optimized model while being faster, the benchmark was run with **this optimization enabled by default**.


| absolute values               | Latency | Memory  | Throghput |
|-------------------------------|---------|---------|-----------|
| base case (bettertransformer) |   19.26 | 8329.2M |      0.42 |
| + fp16                          |   10.32 | 4198.8M |      0.78 |
| + offload                       |   20.46 | 5172.1M |      0.39 |
| + offload + fp16                |   10.91 | 2619.5M |      0.73 |

| Relative value                | Latency | Memory | Throughput |
|-------------------------------|---------|--------|------------|
| + base case (bettertransformer) |      0% |     0% |         0% |
| + fp16                          |    -46% |   -50% |        87% |
| + offload                       |      6% |   -38% |        -6% |
| + offload + fp16                |    -43% |   -69% |       77% |

### Comment

This is where we can see the potential of combining all three optimization features!

The impact of `fp16` on latency is less marked with `batch_size = 1`, but here it is of enormous interest as it can reduce latency by almost half, and almost double throughput!

# Concluding remarks

This notebook tried to showcase a few simple optimization tricks that can greatly improve Bark performance.


* You can use the large version of Bark without any performance degradation and a footprint of just 2GB instead of 5GB, 15% faster, **using 🤗 bettertransformer and cpu offload**.


* Do you prefer high throughput? **Batch by 8 with 🤗 bettertransformer and half-precision**.


* You can get the best of both worlds by using **fp16, 🤗 bettertransformer and cpu offload**!